# Testing different SA methods 3/5
## nltk's vader
TODO attribute the VADER paper here and in the thesis

In [2]:
import csv

from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Ugly hackery, but necessary: stackoverflow.com/questions/4383571/importing-files-from-different-folder
import sys
sys.path.append('../../../')
    
from src.streaming import spark_functions

In [16]:
preprocess = spark_functions.preprocessor()
tokenize = spark_functions.tokenizer()

with open('./../../../data/interim/sanders_hydrated.csv') as csv_file:
    iterator = csv.reader(csv_file, delimiter=',')
    # Load the parts we need and preprocess as well as tokenize the text
    tweets = [(preprocess(text), sentiment) for (topic, sentiment, id, text) in iterator if sentiment=='positive' or sentiment=='negative']

In [19]:
sid = SentimentIntensityAnalyzer()
labeled_correctly = 0
labeled_neutral = 0
for (tweet,sentiment) in tweets:
    ss = sid.polarity_scores(tweet)
    if (ss['neg'] > ss['pos'] and sentiment == 'negative') or (ss['neg'] < ss['pos'] and sentiment == 'positive'):
        labeled_correctly += 1
    elif ss['neg'] == ss['pos']:
        labeled_neutral += 1

labeled_not_wrong = labeled_neutral + labeled_correctly
print("Labeled correctly: %d/%d = %.2d percent" % (labeled_correctly, len(tweets), labeled_correctly/len(tweets)*100))
print("Labeled not wrong: %d/%d = %.2d percent" % (labeled_not_wrong, len(tweets), labeled_not_wrong/len(tweets)*100))

Labeled correctly: 512/946 = 54 percent
Labeled not wrong: 707/946 = 74 percent


In [18]:
# Funny how vader performs way better on the nltk dataset
# TODO look at code and dataset
import nltk.sentiment.util
nltk.sentiment.util.demo_vader_tweets(n_instances=100, output=None)

Loaded 50 tweets
Loaded 50 tweets
Accuracy: 0.85
F-measure [neg]: 0.8235294117647058
F-measure [pos]: 0.8695652173913044
Precision [neg]: 1.0
Precision [pos]: 0.7692307692307693
Recall [neg]: 0.7
Recall [pos]: 1.0
